In [1]:
using Symbolics

In [2]:
# define the variables, q,p in this case
const d = 2

# highest polynomial order to check
const order = 3 

# binomial used to get the combination of variables till the highest order without repeat, nparam = 34 for 3rd order, with z = q,p each of 2 dims
const nparam = binomial(2d + order, order) - 1

34

In [3]:
# # total number of dimensions, where each variable has 2 dimensions
n = 2d

# # verification check for number of variables
1 + n + n*(n+1) ÷ 2 + n*(n+1)*(n+2) ÷ 6 - 1

34

In [4]:
# get variables p and q and store in variable z
@variables q[1:d]
@variables p[1:d]
@variables a[1:nparam]

z = vcat(q,p)

4-element Vector{Num}:
 q[1]
 q[2]
 p[1]
 p[2]

In [5]:
# define empty variable H to store the hamiltonian
H = Num(0)

0

In [6]:
_prod(a, b, c, arrs...) = a .* _prod(b, c, arrs...)
_prod(a, b) = a .* b
_prod(a) = a

function hamiltonian_poly(z, order, inds...)
    ham = []

    if order == 0
        Num(1)
    elseif order == length(inds)
        ham = vcat(ham, _prod([z[i] for i in inds]...))
    else
        start_ind = length(inds) == 0 ? 1 : inds[end]
        for j in start_ind:length(z)
            ham = vcat(ham, hamiltonian_poly(z, order, inds..., j))
        end
    end

    return ham
end

function hamiltonian(z, a, order)
    ham = []

    for i in 1:order
        ham = vcat(ham, hamiltonian_poly(z, i))
    end

    sum(collect(a .* ham))
end

H = hamiltonian(z, a, order)

(p[2]^2)*a[14] + (q[1]^2)*a[5] + (q[2]^2)*a[9] + (p[1]^2)*a[12] + (q[1]^3)*a[15] + (q[2]^3)*a[25] + (p[1]^3)*a[31] + (p[2]^3)*a[34] + a[3]*p[1] + a[2]*q[2] + a[4]*p[2] + a[1]*q[1] + (p[1]^2)*a[32]*p[2] + (q[2]^2)*a[26]*p[1] + (p[1]^2)*a[22]*q[1] + (p[1]^2)*a[28]*q[2] + (p[2]^2)*a[24]*q[1] + (p[2]^2)*a[30]*q[2] + (q[1]^2)*a[17]*p[1] + (q[1]^2)*a[18]*p[2] + (q[2]^2)*a[27]*p[2] + a[13]*p[1]*p[2] + (q[1]^2)*a[16]*q[2] + (q[2]^2)*a[19]*q[1] + (p[2]^2)*a[33]*p[1] + a[7]*p[1]*q[1] + a[10]*p[1]*q[2] + a[8]*p[2]*q[1] + a[11]*p[2]*q[2] + a[6]*q[1]*q[2] + a[20]*p[1]*q[1]*q[2] + a[21]*p[2]*q[1]*q[2] + a[23]*p[1]*p[2]*q[1] + a[29]*p[1]*p[2]*q[2]

In [7]:
# This just puts the terms in an order corresponding to the order of (a) coeffs. we get in the result, so we can compare
ham = []

    for i in 1:order
        ham = vcat(ham, hamiltonian_poly(z, i))
    end

collect(a .* ham)

34-element Vector{Num}:
            a[1]*q[1]
            a[2]*q[2]
            a[3]*p[1]
            a[4]*p[2]
        (q[1]^2)*a[5]
       a[6]*q[1]*q[2]
       a[7]*p[1]*q[1]
       a[8]*p[2]*q[1]
        (q[2]^2)*a[9]
      a[10]*p[1]*q[2]
                    ⋮
  (q[2]^2)*a[26]*p[1]
  (q[2]^2)*a[27]*p[2]
  (p[1]^2)*a[28]*q[2]
 a[29]*p[1]*p[2]*q[2]
  (p[2]^2)*a[30]*q[2]
       (p[1]^3)*a[31]
  (p[1]^2)*a[32]*p[2]
  (p[2]^2)*a[33]*p[1]
       (p[2]^3)*a[34]

In [8]:
Dz = Differential.(z)
∇H = [expand_derivatives(dz(H)) for dz in Dz]

4-element Vector{Num}:
    (p[1]^2)*a[22] + (q[2]^2)*a[19] + (p[2]^2)*a[24] + a[6]*q[2] + a[7]*p[1] + a[8]*p[2] + 2a[5]*q[1] + 3(q[1]^2)*a[15] + a[20]*p[1]*q[2] + a[23]*p[1]*p[2] + a[21]*p[2]*q[2] + 2a[17]*p[1]*q[1] + 2a[18]*p[2]*q[1] + 2a[16]*q[1]*q[2] + a[1]
  (p[1]^2)*a[28] + (p[2]^2)*a[30] + (q[1]^2)*a[16] + a[6]*q[1] + a[10]*p[1] + a[11]*p[2] + 3(q[2]^2)*a[25] + a[29]*p[1]*p[2] + 2a[9]*q[2] + a[20]*p[1]*q[1] + a[21]*p[2]*q[1] + 2a[26]*p[1]*q[2] + 2a[27]*p[2]*q[2] + 2a[19]*q[1]*q[2] + a[2]
 (p[2]^2)*a[33] + (q[1]^2)*a[17] + (q[2]^2)*a[26] + a[7]*q[1] + a[13]*p[2] + a[10]*q[2] + 3(p[1]^2)*a[31] + 2a[12]*p[1] + a[20]*q[1]*q[2] + a[23]*p[2]*q[1] + a[29]*p[2]*q[2] + 2a[32]*p[1]*p[2] + 2a[22]*p[1]*q[1] + 2a[28]*p[1]*q[2] + a[3]
 (q[1]^2)*a[18] + (p[1]^2)*a[32] + (q[2]^2)*a[27] + a[13]*p[1] + a[8]*q[1] + a[11]*q[2] + 2a[14]*p[2] + 3(p[2]^2)*a[34] + a[21]*q[1]*q[2] + a[23]*p[1]*q[1] + a[29]*p[1]*q[2] + 2a[33]*p[1]*p[2] + 2a[24]*p[2]*q[1] + 2a[30]*p[2]*q[2] + a[4]

In [9]:
# line below makes the vector into a hamiltonian by multiplying with the skew-symmetric matrix
f = vcat(∇H[d+1:2d], -∇H[1:d])

4-element Vector{Num}:
 (p[2]^2)*a[33] + (q[1]^2)*a[17] + (q[2]^2)*a[26] + a[7]*q[1] + a[13]*p[2] + a[10]*q[2] + 3(p[1]^2)*a[31] + 2a[12]*p[1] + a[20]*q[1]*q[2] + a[23]*p[2]*q[1] + a[29]*p[2]*q[2] + 2a[32]*p[1]*p[2] + 2a[22]*p[1]*q[1] + 2a[28]*p[1]*q[2] + a[3]
 (q[1]^2)*a[18] + (p[1]^2)*a[32] + (q[2]^2)*a[27] + a[13]*p[1] + a[8]*q[1] + a[11]*q[2] + 2a[14]*p[2] + 3(p[2]^2)*a[34] + a[21]*q[1]*q[2] + a[23]*p[1]*q[1] + a[29]*p[1]*q[2] + 2a[33]*p[1]*p[2] + 2a[24]*p[2]*q[1] + 2a[30]*p[2]*q[2] + a[4]
   -a[1] - (p[1]^2)*a[22] - (q[2]^2)*a[19] - 3(q[1]^2)*a[15] - (p[2]^2)*a[24] - a[6]*q[2] - 2a[5]*q[1] - a[7]*p[1] - a[8]*p[2] - 2a[17]*p[1]*q[1] - a[20]*p[1]*q[2] - a[23]*p[1]*p[2] - a[21]*p[2]*q[2] - 2a[18]*p[2]*q[1] - 2a[16]*q[1]*q[2]
 -a[2] - (p[1]^2)*a[28] - (p[2]^2)*a[30] - (q[1]^2)*a[16] - 3(q[2]^2)*a[25] - a[6]*q[1] - a[10]*p[1] - a[11]*p[2] - 2a[9]*q[2] - a[29]*p[1]*p[2] - 2a[19]*q[1]*q[2] - a[20]*p[1]*q[1] - 2a[26]*p[1]*q[2] - a[21]*p[2]*q[1] - 2a[27]*p[2]*q[2]

In [10]:
H = hamiltonian(z, a, order)

(p[2]^2)*a[14] + (q[1]^2)*a[5] + (q[2]^2)*a[9] + (p[1]^2)*a[12] + (q[1]^3)*a[15] + (q[2]^3)*a[25] + (p[1]^3)*a[31] + (p[2]^3)*a[34] + a[3]*p[1] + a[2]*q[2] + a[4]*p[2] + a[1]*q[1] + (p[1]^2)*a[32]*p[2] + (q[2]^2)*a[26]*p[1] + (p[1]^2)*a[22]*q[1] + (p[1]^2)*a[28]*q[2] + (p[2]^2)*a[24]*q[1] + (p[2]^2)*a[30]*q[2] + (q[1]^2)*a[17]*p[1] + (q[1]^2)*a[18]*p[2] + (q[2]^2)*a[27]*p[2] + a[13]*p[1]*p[2] + (q[1]^2)*a[16]*q[2] + (q[2]^2)*a[19]*q[1] + (p[2]^2)*a[33]*p[1] + a[7]*p[1]*q[1] + a[10]*p[1]*q[2] + a[8]*p[2]*q[1] + a[11]*p[2]*q[2] + a[6]*q[1]*q[2] + a[20]*p[1]*q[1]*q[2] + a[21]*p[2]*q[1]*q[2] + a[23]*p[1]*p[2]*q[1] + a[29]*p[1]*p[2]*q[2]

In [11]:
hamilFunction = build_function(H, z, a)

:(function (ˍ₋arg1, a)
      #= /home/nigelbrucekhan/.julia/packages/SymbolicUtils/pO4d6/src/code.jl:350 =#
      #= /home/nigelbrucekhan/.julia/packages/SymbolicUtils/pO4d6/src/code.jl:351 =#
      #= /home/nigelbrucekhan/.julia/packages/SymbolicUtils/pO4d6/src/code.jl:352 =#
      begin
          (+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((*)((^)(ˍ₋arg1[4], 2), (getindex)(a, 14)), (*)((^)(ˍ₋arg1[1], 2), (getindex)(a, 5))), (*)((^)(ˍ₋arg1[2], 2), (getindex)(a, 9))), (*)((^)(ˍ₋arg1[3], 2), (getindex)(a, 12))), (*)((^)(ˍ₋arg1[1], 3), (getindex)(a, 15))), (*)((^)(ˍ₋arg1[2], 3), (getindex)(a, 25))), (*)((^)(ˍ₋arg1[3], 3), (getindex)(a, 31))), (*)((^)(ˍ₋arg1[4], 3), (getindex)(a, 34))), (*)((getindex)(a, 3), ˍ₋arg1[3])), (*)((getindex)(a, 2), ˍ₋arg1[2])), (*)((getindex)(a, 4), ˍ₋arg1[4])), (*)((getindex)(a, 1), ˍ₋arg1[1])), (*)((*)((^)(ˍ₋arg1[3], 2), (getindex)(a, 32)), ˍ₋arg1[4])), (*)((*)((^)(ˍ₋arg1[

In [12]:
# ˍ₋arg1 = z (p,q) of dims = 2 each 
# a is of size 34. The function below only works for poly order = 3
# t is unused and is only present to comply with ODEProblem syntax
function hamiltonianFunction(ˍ₋arg1, a)
    begin
        (+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((*)((^)(ˍ₋arg1[4], 2), (getindex)(a, 14)), (*)((^)(ˍ₋arg1[1], 2), (getindex)(a, 5))), (*)((^)(ˍ₋arg1[2], 2), (getindex)(a, 9))), (*)((^)(ˍ₋arg1[3], 2), (getindex)(a, 12))), (*)((^)(ˍ₋arg1[1], 3), (getindex)(a, 15))), (*)((^)(ˍ₋arg1[2], 3), (getindex)(a, 25))), (*)((^)(ˍ₋arg1[3], 3), (getindex)(a, 31))), (*)((^)(ˍ₋arg1[4], 3), (getindex)(a, 34))), (*)((getindex)(a, 3), ˍ₋arg1[3])), (*)((getindex)(a, 2), ˍ₋arg1[2])), (*)((getindex)(a, 4), ˍ₋arg1[4])), (*)((getindex)(a, 1), ˍ₋arg1[1])), (*)((*)((^)(ˍ₋arg1[3], 2), (getindex)(a, 32)), ˍ₋arg1[4])), (*)((*)((^)(ˍ₋arg1[2], 2), (getindex)(a, 26)), ˍ₋arg1[3])), (*)((*)((^)(ˍ₋arg1[3], 2), (getindex)(a, 22)), ˍ₋arg1[1])), (*)((*)((^)(ˍ₋arg1[3], 2), (getindex)(a, 28)), ˍ₋arg1[2])), (*)((*)((^)(ˍ₋arg1[4], 2), (getindex)(a, 24)), ˍ₋arg1[1])), (*)((*)((^)(ˍ₋arg1[4], 2), (getindex)(a, 30)), ˍ₋arg1[2])), (*)((*)((^)(ˍ₋arg1[1], 2), (getindex)(a, 17)), ˍ₋arg1[3])), (*)((*)((^)(ˍ₋arg1[1], 2), (getindex)(a, 18)), ˍ₋arg1[4])), (*)((*)((^)(ˍ₋arg1[2], 2), (getindex)(a, 27)), ˍ₋arg1[4])), (*)((*)((getindex)(a, 13), ˍ₋arg1[3]), ˍ₋arg1[4])), (*)((*)((^)(ˍ₋arg1[1], 2), (getindex)(a, 16)), ˍ₋arg1[2])), (*)((*)((^)(ˍ₋arg1[2], 2), (getindex)(a, 19)), ˍ₋arg1[1])), (*)((*)((^)(ˍ₋arg1[4], 2), (getindex)(a, 33)), ˍ₋arg1[3])), (*)((*)((getindex)(a, 7), ˍ₋arg1[3]), ˍ₋arg1[1])), (*)((*)((getindex)(a, 10), ˍ₋arg1[3]), ˍ₋arg1[2])), (*)((*)((getindex)(a, 8), ˍ₋arg1[4]), ˍ₋arg1[1])), (*)((*)((getindex)(a, 11), ˍ₋arg1[4]), ˍ₋arg1[2])), (*)((*)((getindex)(a, 6), ˍ₋arg1[1]), ˍ₋arg1[2])), (*)((*)((*)((getindex)(a, 20), ˍ₋arg1[3]), ˍ₋arg1[1]), ˍ₋arg1[2])), (*)((*)((*)((getindex)(a, 21), ˍ₋arg1[4]), ˍ₋arg1[1]), ˍ₋arg1[2])), (*)((*)((*)((getindex)(a, 23), ˍ₋arg1[3]), ˍ₋arg1[4]), ˍ₋arg1[1])), (*)((*)((*)((getindex)(a, 29), ˍ₋arg1[3]), ˍ₋arg1[4]), ˍ₋arg1[2]))
    end
end

# Lines below are only for testing

# initial test function
x₀ = [2., 0., 0., 0.]

# let a vector be ones initially of length 34 (b/c 34 is number of poly combinations for 2 variables, with 2 dims of highest order 3)
a = ones(34)

# 2 dims each of p and q gives 4 variables
out = zeros(4)

# output of hamiltonFunction stored in out
 HAM = hamiltonianFunction(x₀, a)

14.0

In [13]:
# builds a function that calculates the gradient of the Hamiltonian function
fcode = build_function(f, z, a)[2]

:(function (ˍ₋out, ˍ₋arg1, ˍ₋arg2)
      #= /home/nigelbrucekhan/.julia/packages/SymbolicUtils/pO4d6/src/code.jl:350 =#
      #= /home/nigelbrucekhan/.julia/packages/SymbolicUtils/pO4d6/src/code.jl:351 =#
      #= /home/nigelbrucekhan/.julia/packages/SymbolicUtils/pO4d6/src/code.jl:352 =#
      begin
          begin
              #= /home/nigelbrucekhan/.julia/packages/Symbolics/RMmq8/src/build_function.jl:520 =#
              #= /home/nigelbrucekhan/.julia/packages/SymbolicUtils/pO4d6/src/code.jl:399 =# @inbounds begin
                      #= /home/nigelbrucekhan/.julia/packages/SymbolicUtils/pO4d6/src/code.jl:395 =#
                      ˍ₋out[1] = (+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((*)((^)(ˍ₋arg1[4], 2), (getindex)(a, 33)), (*)((^)(ˍ₋arg1[1], 2), (getindex)(a, 17))), (*)((^)(ˍ₋arg1[2], 2), (getindex)(a, 26))), (*)((getindex)(a, 7), ˍ₋arg1[1])), (*)((getindex)(a, 13), ˍ₋arg1[4])), (*)((getindex)(a, 10), ˍ₋arg1[2])), (*)((*)(3, (^)(ˍ₋arg1[3], 2)), (getindex)(a, 3

In [14]:
# output of fcode used to get function hamilFunction below where ˍ₋arg1 = z (p,q) of dims=2 each. a is of size(34), but only 33 of its elements are used in the gradient, t is unused
function hamilGradient!(ˍ₋out, ˍ₋arg1, a, t)
    begin
        begin
            @inbounds begin
                ˍ₋out[1] = (+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((*)((^)((getindex)(ˍ₋arg1, 4), 2), (getindex)(a, 33)), (*)((^)((getindex)(ˍ₋arg1, 1), 2), (getindex)(a, 17))), (*)((^)((getindex)(ˍ₋arg1, 2), 2), (getindex)(a, 26))), (*)((getindex)(a, 13), (getindex)(ˍ₋arg1, 4))), (*)((getindex)(a, 7), (getindex)(ˍ₋arg1, 1))), (*)((getindex)(a, 10), (getindex)(ˍ₋arg1, 2))), (*)((*)(2, (getindex)(a, 12)), (getindex)(ˍ₋arg1, 3))), (*)((*)(3, (^)((getindex)(ˍ₋arg1, 3), 2)), (getindex)(a, 31))), (*)((*)((getindex)(a, 20), (getindex)(ˍ₋arg1, 1)), (getindex)(ˍ₋arg1, 2))), (*)((*)((getindex)(a, 23), (getindex)(ˍ₋arg1, 4)), (getindex)(ˍ₋arg1, 1))), (*)((*)((getindex)(a, 29), (getindex)(ˍ₋arg1, 4)), (getindex)(ˍ₋arg1, 2))), (*)((*)((*)(2, (getindex)(a, 22)), (getindex)(ˍ₋arg1, 3)), (getindex)(ˍ₋arg1, 1))), (*)((*)((*)(2, (getindex)(a, 32)), (getindex)(ˍ₋arg1, 3)), (getindex)(ˍ₋arg1, 4))), (*)((*)((*)(2, (getindex)(a, 28)), (getindex)(ˍ₋arg1, 3)), (getindex)(ˍ₋arg1, 2))), (getindex)(a, 3))
                ˍ₋out[2] = (+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((*)((^)((getindex)(ˍ₋arg1, 1), 2), (getindex)(a, 18)), (*)((^)((getindex)(ˍ₋arg1, 3), 2), (getindex)(a, 32))), (*)((^)((getindex)(ˍ₋arg1, 2), 2), (getindex)(a, 27))), (*)((getindex)(a, 13), (getindex)(ˍ₋arg1, 3))), (*)((getindex)(a, 8), (getindex)(ˍ₋arg1, 1))), (*)((getindex)(a, 11), (getindex)(ˍ₋arg1, 2))), (*)((*)(3, (^)((getindex)(ˍ₋arg1, 4), 2)), (getindex)(a, 34))), (*)((*)(2, (getindex)(a, 14)), (getindex)(ˍ₋arg1, 4))), (*)((*)((getindex)(a, 21), (getindex)(ˍ₋arg1, 1)), (getindex)(ˍ₋arg1, 2))), (*)((*)((getindex)(a, 23), (getindex)(ˍ₋arg1, 3)), (getindex)(ˍ₋arg1, 1))), (*)((*)((getindex)(a, 29), (getindex)(ˍ₋arg1, 3)), (getindex)(ˍ₋arg1, 2))), (*)((*)((*)(2, (getindex)(a, 33)), (getindex)(ˍ₋arg1, 3)), (getindex)(ˍ₋arg1, 4))), (*)((*)((*)(2, (getindex)(a, 24)), (getindex)(ˍ₋arg1, 4)), (getindex)(ˍ₋arg1, 1))), (*)((*)((*)(2, (getindex)(a, 30)), (getindex)(ˍ₋arg1, 4)), (getindex)(ˍ₋arg1, 2))), (getindex)(a, 4))
                ˍ₋out[3] = (+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((*)(-1, (getindex)(a, 1)), (*)((*)(-1, (^)((getindex)(ˍ₋arg1, 3), 2)), (getindex)(a, 22))), (*)((*)(-1, (^)((getindex)(ˍ₋arg1, 4), 2)), (getindex)(a, 24))), (*)((*)(-1, (^)((getindex)(ˍ₋arg1, 2), 2)), (getindex)(a, 19))), (*)((*)(-2, (getindex)(a, 5)), (getindex)(ˍ₋arg1, 1))), (*)((*)(-1, (getindex)(a, 7)), (getindex)(ˍ₋arg1, 3))), (*)((*)(-1, (getindex)(a, 8)), (getindex)(ˍ₋arg1, 4))), (*)((*)(-1, (getindex)(a, 6)), (getindex)(ˍ₋arg1, 2))), (*)((*)(-3, (^)((getindex)(ˍ₋arg1, 1), 2)), (getindex)(a, 15))), (*)((*)((*)(-2, (getindex)(a, 17)), (getindex)(ˍ₋arg1, 3)), (getindex)(ˍ₋arg1, 1))), (*)((*)((*)(-2, (getindex)(a, 18)), (getindex)(ˍ₋arg1, 4)), (getindex)(ˍ₋arg1, 1))), (*)((*)((*)(-2, (getindex)(a, 16)), (getindex)(ˍ₋arg1, 1)), (getindex)(ˍ₋arg1, 2))), (*)((*)((*)(-1, (getindex)(a, 20)), (getindex)(ˍ₋arg1, 3)), (getindex)(ˍ₋arg1, 2))), (*)((*)((*)(-1, (getindex)(a, 21)), (getindex)(ˍ₋arg1, 4)), (getindex)(ˍ₋arg1, 2))), (*)((*)((*)(-1, (getindex)(a, 23)), (getindex)(ˍ₋arg1, 3)), (getindex)(ˍ₋arg1, 4)))
                ˍ₋out[4] = (+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((+)((*)(-1, (getindex)(a, 2)), (*)((*)(-1, (^)((getindex)(ˍ₋arg1, 1), 2)), (getindex)(a, 16))), (*)((*)(-1, (^)((getindex)(ˍ₋arg1, 3), 2)), (getindex)(a, 28))), (*)((*)(-1, (^)((getindex)(ˍ₋arg1, 4), 2)), (getindex)(a, 30))), (*)((*)(-1, (getindex)(a, 10)), (getindex)(ˍ₋arg1, 3))), (*)((*)(-1, (getindex)(a, 6)), (getindex)(ˍ₋arg1, 1))), (*)((*)(-1, (getindex)(a, 11)), (getindex)(ˍ₋arg1, 4))), (*)((*)(-2, (getindex)(a, 9)), (getindex)(ˍ₋arg1, 2))), (*)((*)(-3, (^)((getindex)(ˍ₋arg1, 2), 2)), (getindex)(a, 25))), (*)((*)((*)(-2, (getindex)(a, 19)), (getindex)(ˍ₋arg1, 1)), (getindex)(ˍ₋arg1, 2))), (*)((*)((*)(-1, (getindex)(a, 20)), (getindex)(ˍ₋arg1, 3)), (getindex)(ˍ₋arg1, 1))), (*)((*)((*)(-1, (getindex)(a, 21)), (getindex)(ˍ₋arg1, 4)), (getindex)(ˍ₋arg1, 1))), (*)((*)((*)(-2, (getindex)(a, 26)), (getindex)(ˍ₋arg1, 3)), (getindex)(ˍ₋arg1, 2))), (*)((*)((*)(-2, (getindex)(a, 27)), (getindex)(ˍ₋arg1, 4)), (getindex)(ˍ₋arg1, 2))), (*)((*)((*)(-1, (getindex)(a, 29)), (getindex)(ˍ₋arg1, 3)), (getindex)(ˍ₋arg1, 4)))
                nothing
            end
        end
    end
end

# Lines below are only for testing

# initial test function
x₀ = [2., 0., 0., 0.]

# let a vector be ones initially of length 34 (b/c 34 is number of poly combinations for 2 variables, with 2 dims of highest order 3)
a = ones(34)

# 2 dims each of p and q gives 4 variables
out = zeros(4)

# output of hamilGradient! stored in out
 t= 0
 hamilGradient!(out, x₀, a, t)

In [15]:
# constructs an augmented library of candidate nonlinear functions
_prod(a, b, c, arrs...) = a .* _prod(b, c, arrs...)
_prod(a, b) = a .* b
_prod(a) = a

function hamiltonian_poly_combos(z, order, inds...)
    ham = []

    if order == 0
        Num(1)
    elseif order == length(inds)
        ham = vcat(ham, _prod([z[i] for i in inds]...))
    else
        start_ind = length(inds) == 0 ? 1 : inds[end]
        for j in start_ind:length(z)
            ham = vcat(ham, hamiltonian_poly_combos(z, order, inds..., j))
        end
    end

    return ham
end

function hamiltonian_basis_maker(z, order)
    ham = []

    for i in 1:order
        ham = vcat(ham, hamiltonian_poly_combos(z, i))
    end

    hcat(ham)
end
H = hamiltonian_basis_maker(z, order)

34×1 Matrix{Num}:
           q[1]
           q[2]
           p[1]
           p[2]
         q[1]^2
      q[1]*q[2]
      p[1]*q[1]
      p[2]*q[1]
         q[2]^2
      p[1]*q[2]
              ⋮
  (q[2]^2)*p[1]
  (q[2]^2)*p[2]
  (p[1]^2)*q[2]
 p[1]*p[2]*q[2]
  (p[2]^2)*q[2]
         p[1]^3
  (p[1]^2)*p[2]
  (p[2]^2)*p[1]
         p[2]^3

In [16]:
# builds a function that constructs an augmented library of candidate nonlinear functions
θ = build_function(H, z, order)

(:(function (ˍ₋arg1, 3)
      #= /home/nigelbrucekhan/.julia/packages/SymbolicUtils/pO4d6/src/code.jl:350 =#
      #= /home/nigelbrucekhan/.julia/packages/SymbolicUtils/pO4d6/src/code.jl:351 =#
      #= /home/nigelbrucekhan/.julia/packages/SymbolicUtils/pO4d6/src/code.jl:352 =#
      begin
          begin
              #= /home/nigelbrucekhan/.julia/packages/SymbolicUtils/pO4d6/src/code.jl:445 =#
              (SymbolicUtils.Code.create_array)(typeof(ˍ₋arg1), nothing, Val{2}(), Val{(34, 1)}(), ˍ₋arg1[1], ˍ₋arg1[2], ˍ₋arg1[3], ˍ₋arg1[4], (^)(ˍ₋arg1[1], 2), (*)(ˍ₋arg1[1], ˍ₋arg1[2]), (*)(ˍ₋arg1[3], ˍ₋arg1[1]), (*)(ˍ₋arg1[4], ˍ₋arg1[1]), (^)(ˍ₋arg1[2], 2), (*)(ˍ₋arg1[3], ˍ₋arg1[2]), (*)(ˍ₋arg1[4], ˍ₋arg1[2]), (^)(ˍ₋arg1[3], 2), (*)(ˍ₋arg1[3], ˍ₋arg1[4]), (^)(ˍ₋arg1[4], 2), (^)(ˍ₋arg1[1], 3), (*)((^)(ˍ₋arg1[1], 2), ˍ₋arg1[2]), (*)((^)(ˍ₋arg1[1], 2), ˍ₋arg1[3]), (*)((^)(ˍ₋arg1[1], 2), ˍ₋arg1[4]), (*)((^)(ˍ₋arg1[2], 2), ˍ₋arg1[1]), (*)((*)(ˍ₋arg1[3], ˍ₋arg1[1]), ˍ₋arg1[2]), (*)((*)(ˍ₋arg1[4]